# Read MODIS Terra/Aqua netcdf as xarray
- HDF4 MODIS was converted to netCDF using nccopy 

In [ ]:
pip install cmaps

In [ ]:
import xarray as xr
import s3fs
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmaps

xr.set_options(display_style='html')
%matplotlib inline

## Connect to bucket (anonymous login for public data only)

In [ ]:
fs = s3fs.S3FileSystem(anon=True,
      client_kwargs={
         'endpoint_url': 'https://object-store.cloud.muni.cz'
      })

In [ ]:
remote_files = fs.ls('MODIS')

In [ ]:
remote_files[:10]

In [ ]:
# Iterate through remote_files to create a fileset
fileset = [fs.open(file) for file in remote_files]

## Create time index from filename to concatenate netCDF files along time dimension

In [ ]:
def paths_to_datetimeindex(paths):
    return  [datetime.strptime(date.split('.A')[-1].split('.')[0], '%Y%j') for date in paths]

In [ ]:
# Create variable used for time axis
time_var = xr.Variable('time', paths_to_datetimeindex(remote_files))

In [ ]:
time_var

## Concatenate all files

In [ ]:
# Load in and concatenate all individual GeoTIFFs
dset = xr.concat([xr.open_mfdataset([i],) for i in fileset],
                        dim=time_var)

In [ ]:
dset = dset.rename_dims({'YDim:mod08': 'latitude', 'XDim:mod08':'longitude', 'Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08':'levels'})

In [ ]:
dset = dset.rename_vars({'YDim':'latitude', 'XDim':'longitude', 'Effective_Optical_Depth_Average_Ocean_Micron_Levels': 'levels'})

In [ ]:
x = dset.isel(time=0).longitude.squeeze().reset_coords(drop=True)
y = dset.isel(time=0).latitude.squeeze().reset_coords(drop=True)
z = dset.isel(time=0).levels.squeeze().reset_coords(drop=True)

In [ ]:
dset = dset.assign_coords({"longitude": x, "latitude": y, 'levels': z})

In [ ]:
dset

## Visualize one single date

In [ ]:
fig=plt.figure(figsize=(17,10))
# Define the projection
crs=ccrs.PlateCarree()

# We're using cartopy and are plotting in Orthographic projection 
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.Mercator(central_longitude=12.0))
ax.coastlines(resolution='10m')

# We need to project our data to the new Mercator projection and for this we use `transform`.
# we set the original data projection in transform (here PlateCarree)
# we only plot values greather than 0 
dset['Aerosol_Optical_Depth_Land_Ocean_Mean_Mean'].isel(time=0).plot(ax=ax, transform=ccrs.PlateCarree(), cmap=cmaps.BlueYellowRed)
# Title for plot
plt.title('Aerosol_Optical_Depth_Land_Ocean_Mean_Mean\n',fontsize = 16, fontweight = 'bold', pad=10)
plt.savefig('Aerosol_Optical_Depth_Land_Ocean_Mean_Mean.png')

## Save results into local netCDF file

In [ ]:
dset.to_netcdf('MOD08_M3_20000201-20210901.nc')